# Instalar libs

In [2]:
# !pip install memoizit
# !pip install memoizit[redis]
# !pip install redis
# !pip install redis[hiredis]

# Importar Configurar Memoizit + Redis

In [127]:
import redis
import time
import random
from memoizit import Memoizer
from itertools import combinations
from IPython.display import display_markdown
import numpy as np

m = Memoizer(
    backend="redis",
    host="localhost",
    port="6379"    
)

In [4]:
# @m.memoize(
#     expiration=60*60*24
# )
def knapsack(capacidade_maxima, pesos, valores, n):
    # Inicializando uma matriz para armazenar os valores máximos possíveis
    # para diferentes capacidades e número de itens
    dp = [[0] * (capacidade_maxima + 1) for _ in range(n + 1)] # Gera n + 1 elementos contendo array de zeros de 0 até capacidade máxima

    # Preenchendo a matriz dp usando a abordagem de programação dinâmica
    for i in range(1, n + 1):
        peso_item = pesos[i - 1]
        valor_item = valores[i - 1]
        for capacidade_atual in range(1, capacidade_maxima + 1):
            if peso_item <= capacidade_atual:
                # Se o peso do item for menor ou igual à capacidade atual,
                # podemos considerar incluir o item na mochila
                dp[i][capacidade_atual] = max(dp[i - 1][capacidade_atual],
                                              dp[i - 1][capacidade_atual - peso_item] + valor_item)
            else:
                # Se o peso do item for maior que a capacidade atual,
                # não podemos incluir o item na mochila
                dp[i][capacidade_atual] = dp[i - 1][capacidade_atual]

    # O valor máximo estará na célula dp[n][capacidade_maxima]
    valor_maximo = dp[n][capacidade_maxima]

    # Reconstruindo a lista de itens selecionados
    itens_selecionados = []
    capacidade_restante = capacidade_maxima
    for i in range(n, 0, -1):
        if dp[i][capacidade_restante] != dp[i - 1][capacidade_restante]:
            itens_selecionados.append(i - 1)
            capacidade_restante -= pesos[i - 1]

    return valor_maximo, itens_selecionados, dp


# Exemplo padrão

In [28]:
# Exemplo de uso
items = [
    (15, 15, 'Saco de dormir'),
    (3, 7, 'Corda'),
    (2, 10, 'Canivete'),
    (5, 5, 'Tocha'),
    (9, 8, 'Garrafa'),
    (20, 17, 'Comida'),
]

capacidade_maxima = 30
pesos = [_[0] for _ in items]
valores = [_[1] for _ in items]
n = len(pesos)

# Inicia a contagem do tempo
start_time = time.time()

max_valor, itens_selecionados, itens_analisados = knapsack(capacidade_maxima, pesos, valores, n)

# Finaliza a contagem do tempo
end_time = time.time()

# Calcula o tempo de execução
execution_time = end_time - start_time

# Exibe o tempo de execução
print("Tempo de execução:", execution_time, "segundos")

print("Valor máximo:", max_valor)
print("Itens selecionados:", itens_selecionados)



Tempo de execução: 0.0 segundos
Valor máximo: 40
Itens selecionados: [4, 2, 1, 0]


In [129]:
data = []
combinacoes = [list(combinations(map(lambda x: x[2], items), _)) for _ in range(1, len(items) + 1)]
peso_valor_produtos = { _[2] : { "peso": _[0], "valor": _[1] } for _ in items }
produtos_selecionados = [items[i][2] for i in itens_selecionados]

output = "# Combinação Produtos"

for combinacao in enumerate(combinacoes):
   id_combinacao, combinacoes_produtos = combinacao
   
   for cp in enumerate(combinacoes_produtos):
      id_combinacao_produto, produtos = cp
      id_combinacao_produto_chave = f"{id_combinacao:03d}{id_combinacao_produto:03d}"

      for p in produtos:
         data.append(
            (
               p,
               id_combinacao_produto_chave,
            )
         )

cenarios = list(set([_ for _ in map(lambda i: i[1], data)]))
cenarios.sort()

produtos = list(set([_ for _ in map(lambda i: i[0], data)]))
produtos.sort()

matriz = [[p] + [
   (c, 
    True if p in map(lambda i: i[0], filter(lambda i: i[1] == c, data)) else False) 
    for c in cenarios] 
    for p in produtos]

total = np.array([[0] * len(cenarios)] * 2)

output += "\n|" + "|".join([" Produto "] + [f" C_{c[0]} " for c in enumerate(cenarios)]) + "|"
output += "\n|" + "|".join(["---------"] + ["-".join(([''] * (len(c) + 4))) for c in enumerate(cenarios)]) + "|"

for m in matriz:
   peso, valor = peso_valor_produtos[m[0]]["peso"], peso_valor_produtos[m[0]]["valor"]
   output += "\n|" + "|".join([m[0]] + [str(peso) if i[1] else "" for i in m[1:]]) + "|"

   total_peso = [peso if i[1] else 0 for i in m[1:]]
   total_valor = [valor if i[1] else 0 for i in m[1:]]

   total = total + np.array([total_peso, total_valor])

output += "\n|" + "|".join(["<b>Peso</b>"] + [str(i) for i in total[0]]) + "|"
output += "\n|" + "|".join(["<b>Valor</b>"] + [str(i) for i in total[1]]) + "|"

display_markdown(output, raw=True)


# Combinação Produtos
| Produto | C_0 | C_1 | C_2 | C_3 | C_4 | C_5 | C_6 | C_7 | C_8 | C_9 | C_10 | C_11 | C_12 | C_13 | C_14 | C_15 | C_16 | C_17 | C_18 | C_19 | C_20 | C_21 | C_22 | C_23 | C_24 | C_25 | C_26 | C_27 | C_28 | C_29 | C_30 | C_31 | C_32 | C_33 | C_34 | C_35 | C_36 | C_37 | C_38 | C_39 | C_40 | C_41 | C_42 | C_43 | C_44 | C_45 | C_46 | C_47 | C_48 | C_49 | C_50 | C_51 | C_52 | C_53 | C_54 | C_55 | C_56 | C_57 | C_58 | C_59 | C_60 | C_61 | C_62 |
|---------|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|
|Canivete|||2|||||2||||2||||2|2|2||||2||||2|2|2||||2|2|2||||2|2|2||2|2|2||||2|2|2||2|2|2||2|2|2|2||2|2|2|
|Comida||||||20|||||20||||20|||20||20|20||||20|||20||20|20|||20||20|20||20|20|20|||20||20|20||20|20|20||20|20|20|20||20|20|20|20|20|20|
|Corda||3|||||3|||||3|3|3|3|||||||3|3|3|3|||||||3|3|3|3|3|3|||||3|3|3|3|3|3|||||3|3|3|3||3|3|3|3||3|3|
|Garrafa|||||9|||||9||||9|||9||9||9|||9|||9||9||9||9||9||9|9||9|9||9||9||9|9||9|9|9||9|9|9|9||9|9|9|9|9|
|Saco de dormir|15||||||15|15|15|15|15|||||||||||15|15|15|15|15|15|15|15|15|15|||||||||||15|15|15|15|15|15|15|15|15|15||||||15|15|15|15|15||15|
|Tocha||||5|||||5||||5|||5|||5|5|||5|||5|||5|5||5|||5|5||5|5||5|5|||5|5||5|5||5|5|5||5|5|5|5||5|5|5|5|
|<b>Peso</b>|15|3|2|5|9|20|18|17|20|24|35|5|8|12|23|7|11|22|14|25|29|20|23|27|38|22|26|37|29|40|44|10|14|25|17|28|32|16|27|31|34|25|29|40|32|43|47|31|42|46|49|19|30|34|37|36|34|45|49|52|51|39|54|
|<b>Valor</b>|15|7|10|5|8|17|22|25|20|23|32|17|12|15|24|15|18|27|13|22|25|32|27|30|39|30|33|42|28|37|40|22|25|34|20|29|32|23|32|35|30|37|40|49|35|44|47|38|47|50|45|30|39|42|37|40|45|54|57|52|55|47|62|

In [124]:
x_total = np.array(total)

In [126]:
x_total + x_total

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  4,  0,  0,  0,  0,  4,  0,  0,  0,  4,  0,  0,  0,  4,  4,
         4,  0,  0,  0,  4,  0,  0,  0,  4,  4,  4,  0,  0,  0,  4,  4,
         4,  0,  0,  0,  4,  4,  4,  0,  4,  4,  4,  0,  0,  0,  4,  4,
         4,  0,  4,  4,  4,  0,  4,  4,  4,  4,  0,  4,  4,  4,  0,  0,
         0,  0,  0, 40,  0,  0,  0,  0, 40,  0,  0,  0, 40,  0,  0, 40,
         0, 40, 40,  0,  0,  0, 40,  0,  0, 40,  0, 40, 40,  0,  0, 40,
         0, 40, 40,  0, 40, 40, 40,  0,  0, 40,  0, 40, 40,  0, 40, 40,
        40,  0, 40, 40, 40, 40,  0, 40, 40, 40, 40, 40, 40,  0,  6,  0,
         0,  0,  0,  6,  0,  0,  0,  0,  6,  6,  6,  6,  0,  0,  0,  0,
         0,  0,  6,  6,  6,  6,  0,  0,  0,  0,  0,  0,  6,  6, 